In [ ]:
import pandas as pd
import numpy as np
#from pyproj import Transformer
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Group points by time window (1-hour threshold)
time_threshold = 3600  # 1 hour in seconds

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Define a function to project points onto the Earth sphere surface
def project_to_ecef(x, y, z, radius=6371e3):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm) * radius, (y / norm) * radius, (z / norm) * radius

# Step 6: Create the Earth sphere for reference
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.3, colorscale='Blues', showscale=False, name='Earth'
)

# Step 7: Create frames for animation with projections on the sphere
frames = []

for i, group in enumerate(groups):
    # Original points
    group_data = df.iloc[group]
    x, y, z = group_data['X'], group_data['Y'], group_data['Z']

    # Projected points on the ECEF sphere
    proj_x, proj_y, proj_z = project_to_ecef(x, y, z)

    # Create a frame with both original and projected points
    frame_data = [
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Original Group {i}"
        ),
        go.Scatter3d(
            x=proj_x, y=proj_y, z=proj_z,
            mode='markers',
            marker=dict(size=4, color='green', opacity=0.6),
            name=f"Projected Group {i}"
        )
    ]

    frames.append(go.Frame(data=frame_data, name=f"Group {i}"))

# Step 8: Layout configuration with Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points and ECEF Projections on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 9: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[earth], layout=layout, frames=frames)

# Step 10: Save and open the plot
fig.write_html("satellite_ecef_projection.html", auto_open=True)